In [ ]:
import scanpy as sc
import scanpy.external as sce
import pandas as pd
import numpy as np
import os
import triku as tk
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
import ray
import subprocess
from scipy.sparse import csr_matrix
from IPython.display import display, HTML

from tqdm.notebook import tqdm

from bokeh.io import show, output_notebook, reset_output

reset_output()
output_notebook()

# Loading full datasets

In [ ]:
adata_oprescu_d0 = sc.read('data/processed/oprescu_d0.h5')

In [ ]:
adata_scott_d0 = sc.read('data/processed/scott_d0.h5')

In [ ]:
adata_de_micheli_mouse_d0 = sc.read('data/processed/de_micheli_mouse_d0.h5')

In [ ]:
adata_giordani = sc.read('data/processed/giordani.h5')

In [ ]:
adata_proietti = sc.read('data/processed/proietti.h5')

In [ ]:
adata_ronzoni = sc.read('data/processed/ronzoni.h5')

In [ ]:
adata_dellorso = sc.read('data/processed/dellorso.h5')

In [ ]:
# Move the UMAPs
w = -1.4

list_new_coords = []
for adata in [adata_oprescu_d0, adata_scott_d0, adata_de_micheli_mouse_d0,
              adata_giordani, adata_proietti, adata_ronzoni, adata_dellorso]:
    X_coords = (adata.obsm['X_umap'][:, 0] - adata.obsm['X_umap'][:, 0].min()) / (adata.obsm['X_umap'][:, 0].max() - adata.obsm['X_umap'][:, 0].min())
    Y_coords = (adata.obsm['X_umap'][:, 1] - adata.obsm['X_umap'][:, 1].min()) / (adata.obsm['X_umap'][:, 1].max() - adata.obsm['X_umap'][:, 1].min())
    list_new_coords.append(np.array([X_coords, Y_coords]).transpose())
    
adata_oprescu_d0.obsm['X_umap'] = (list_new_coords[0] + np.array([[0, 0]])).astype(np.float32)
adata_scott_d0.obsm['X_umap'] = (list_new_coords[1] + np.array([[0, w]])).astype(np.float32)
adata_de_micheli_mouse_d0.obsm['X_umap'] = (list_new_coords[2] + np.array([[0, 2*w]])).astype(np.float32)
adata_giordani.obsm['X_umap'] = (list_new_coords[3] + np.array([[-w, 0]])).astype(np.float32)
adata_proietti.obsm['X_umap'] = (list_new_coords[4] + np.array([[-w, w]])).astype(np.float32)
adata_ronzoni.obsm['X_umap'] = (list_new_coords[5] + np.array([[-w, 2 * w]])).astype(np.float32)
adata_dellorso.obsm['X_umap'] = (list_new_coords[6] + np.array([[-2*w, 0]])).astype(np.float32)


In [ ]:
adata_joined = sc.AnnData.concatenate(adata_oprescu_d0, adata_scott_d0, adata_de_micheli_mouse_d0,
              adata_giordani, adata_proietti, adata_ronzoni, adata_dellorso)

In [ ]:
adata_joined.obs = adata_joined.obs[['cell_type', 'dataset', 'krano_type', 'condition']]
del adata_joined.var
adata_joined.uns['krano_type_colors'] = ['#bcbcbc', '#900C3F', '#286e87']
adata_joined.uns['condition_colors'] = ['#900C3F', '#286e87', '#bcbcbc', ]

In [ ]:
sc.pl.umap(adata_joined, color=['krano_type', 'condition', 'dataset',])

In [ ]:
adata_joined

In [ ]:
adata_joined.write_h5ad('data/processed/joined.h5')

# Loading sub datasets

In [ ]:
adata_oprescu_d0_sub = sc.read('data/processed/oprescu_d0_sub.h5')

In [ ]:
adata_scott_d0_sub = sc.read('data/processed/scott_d0_sub.h5')

In [ ]:
adata_de_micheli_mouse_d0_sub = sc.read('data/processed/de_micheli_mouse_d0_sub.h5')

In [ ]:
adata_giordani_sub = sc.read('data/processed/giordani_sub.h5')

In [ ]:
adata_proietti_sub = sc.read('data/processed/proietti_sub.h5')

In [ ]:
adata_ronzoni_sub = sc.read('data/processed/ronzoni_sub.h5')

In [ ]:
adata_dellorso_sub = sc.read('data/processed/dellorso_sub.h5')

In [ ]:
# Move the UMAPs
w = -1.4

list_new_coords = []
for adata in [adata_oprescu_d0_sub, adata_scott_d0_sub, adata_de_micheli_mouse_d0_sub,
              adata_giordani_sub, adata_proietti_sub, adata_ronzoni_sub, adata_dellorso_sub]:
    X_coords = (adata.obsm['X_umap'][:, 0] - adata.obsm['X_umap'][:, 0].min()) / (adata.obsm['X_umap'][:, 0].max() - adata.obsm['X_umap'][:, 0].min())
    Y_coords = (adata.obsm['X_umap'][:, 1] - adata.obsm['X_umap'][:, 1].min()) / (adata.obsm['X_umap'][:, 1].max() - adata.obsm['X_umap'][:, 1].min())
    list_new_coords.append(np.array([X_coords, Y_coords]).transpose())
    
adata_oprescu_d0_sub.obsm['X_umap'] = (list_new_coords[0] + np.array([[0, 0]])).astype(np.float32)
adata_scott_d0_sub.obsm['X_umap'] = (list_new_coords[1] + np.array([[0, w]])).astype(np.float32)
adata_de_micheli_mouse_d0_sub.obsm['X_umap'] = (list_new_coords[2] + np.array([[0, 2*w]])).astype(np.float32)
adata_giordani_sub.obsm['X_umap'] = (list_new_coords[3] + np.array([[-w, 0]])).astype(np.float32)
adata_proietti_sub.obsm['X_umap'] = (list_new_coords[4] + np.array([[-w, w]])).astype(np.float32)
adata_ronzoni_sub.obsm['X_umap'] = (list_new_coords[5] + np.array([[-w, 2 * w]])).astype(np.float32)
adata_dellorso_sub.obsm['X_umap'] = (list_new_coords[6] + np.array([[-2*w, 0]])).astype(np.float32)


In [ ]:
adata_joined_sub = sc.AnnData.concatenate(adata_oprescu_d0_sub, adata_scott_d0_sub, adata_de_micheli_mouse_d0_sub,
              adata_giordani_sub, adata_proietti_sub, adata_ronzoni, adata_dellorso_sub)

In [ ]:
adata_joined_sub.obs = adata_joined_sub.obs[['cell_type', 'dataset', 'krano_type', 'condition']]
del adata_joined_sub.var
adata_joined_sub.uns['krano_type_colors'] = ['#bcbcbc', '#900C3F', '#286e87']
adata_joined_sub.uns['condition_colors'] = ['#900C3F', '#286e87', '#bcbcbc', ]

In [ ]:
sc.pl.umap(adata_joined_sub, color=['krano_type', 'condition', 'dataset',])

In [ ]:
adata_joined_sub

In [ ]:
adata_joined_sub.write_h5ad('data/processed/joined_sub.h5')